In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt

#### Set up device agnostic code

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

#### Retrive the dataset. Apply transformations and make dataloaders

In [17]:
data_path = Path("data")
train_dir = data_path / "train"
test_dir = data_path / "test"

In [18]:
IMAGE_SIZE = 180
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomCrop(40, pad_if_needed=True),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
])

In [19]:
train_data = datasets.ImageFolder(root=train_dir,
                                  transform=train_transform,
                                  target_transform=None)

test_data = datasets.ImageFolder(root=test_dir,
                                 transform=test_transform)

In [20]:
class_names = train_data.classes
class_dict = train_data.class_to_idx
class_names, class_dict

(['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'],
 {'angry': 0,
  'disgust': 1,
  'fear': 2,
  'happy': 3,
  'neutral': 4,
  'sad': 5,
  'surprise': 6})

In [21]:
len(train_data), len(test_data)

(28709, 7178)

In [22]:
BATCH_SIZE = 48
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=6)

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False,
                             num_workers=6)

#### Initialize the model with pretrained weights and change the final output shape to match the number of emotions.

In [23]:
import torchvision
model = torchvision.models.efficientnet_b0(weights=torchvision.models.EfficientNet_B0_Weights)

/home/toconn23/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
from torchinfo import summary
summary(model=model,
        input_size=(BATCH_SIZE, 3, IMAGE_SIZE, IMAGE_SIZE),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [48, 3, 180, 180]    [48, 1000]           --                   True
├─Sequential (features)                                      [48, 3, 180, 180]    [48, 1280, 6, 6]     --                   True
│    └─Conv2dNormActivation (0)                              [48, 3, 180, 180]    [48, 32, 90, 90]     --                   True
│    │    └─Conv2d (0)                                       [48, 3, 180, 180]    [48, 32, 90, 90]     864                  True
│    │    └─BatchNorm2d (1)                                  [48, 32, 90, 90]     [48, 32, 90, 90]     64                   True
│    │    └─SiLU (2)                                         [48, 32, 90, 90]     [48, 32, 90, 90]     --                   --
│    └─Sequential (1)                                        [48, 32, 90, 90]     [48, 16, 90,

#### Change the output of the last layer to the amount of classes

In [25]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

output_shape = len(class_names)
model.classifier = torch.nn.Sequential(nn.Dropout(p=0.70, inplace=False),
                                       nn.Linear(in_features=1280,
                                                       out_features=output_shape,
                                                       bias=True)).to(device)

In [26]:
summary(model=model,
        input_size=(BATCH_SIZE, 3, IMAGE_SIZE, IMAGE_SIZE),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [48, 3, 180, 180]    [48, 7]              --                   True
├─Sequential (features)                                      [48, 3, 180, 180]    [48, 1280, 6, 6]     --                   True
│    └─Conv2dNormActivation (0)                              [48, 3, 180, 180]    [48, 32, 90, 90]     --                   True
│    │    └─Conv2d (0)                                       [48, 3, 180, 180]    [48, 32, 90, 90]     864                  True
│    │    └─BatchNorm2d (1)                                  [48, 32, 90, 90]     [48, 32, 90, 90]     64                   True
│    │    └─SiLU (2)                                         [48, 32, 90, 90]     [48, 32, 90, 90]     --                   --
│    └─Sequential (1)                                        [48, 32, 90, 90]     [48, 16, 90,

#### Define loss function and optimizer

In [27]:
# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

#### Train the model

In [28]:
import train_lib
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)
# Start the timer
from timeit import default_timer as timer 
start_time = timer()

# Setup training and save the results
results = train_lib.train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=10,
                       device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

 20%|██        | 1/5 [01:17<05:09, 77.47s/it]

Epoch: 1 | train_loss: 1.2403 | train_acc: 0.5325 | test_loss: 1.0662 | test_acc: 0.5978


 40%|████      | 2/5 [02:32<03:48, 76.24s/it]

Epoch: 2 | train_loss: 1.0132 | train_acc: 0.6230 | test_loss: 0.9840 | test_acc: 0.6358


 60%|██████    | 3/5 [03:51<02:34, 77.18s/it]

Epoch: 3 | train_loss: 0.9205 | train_acc: 0.6566 | test_loss: 1.0104 | test_acc: 0.6339


 80%|████████  | 4/5 [05:08<01:17, 77.39s/it]

Epoch: 4 | train_loss: 0.8624 | train_acc: 0.6794 | test_loss: 0.9830 | test_acc: 0.6447


100%|██████████| 5/5 [06:27<00:00, 77.54s/it]

Epoch: 5 | train_loss: 0.8053 | train_acc: 0.7012 | test_loss: 0.9874 | test_acc: 0.6540
Total training time: 387.719 seconds


#### Save the model

In [29]:
from pathlib import Path
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True,
                 exist_ok=True)
MODEL_NAME = "Model.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME 
torch.save(obj=model,
           f=MODEL_SAVE_PATH)
